# Assignment 06
**Author**: Crista Barnes

**Date**: 3/12/2021

## Retrieve data from webpage 

In [1]:
import pandas as pd
htmltables=pd.read_html("https://www.genealogybranches.com/censuscosts.html")

## Determine and define the data of interest 

In [2]:
len(htmltables)


2

In [3]:
df1=htmltables[0]
df1

,0,1,2,3
0,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents
6,1840,17069458,"$833,371",4.88 cents
7,1850,23191876,"$1,423,351",6.14 cents
8,1860,31443321,"$1,969,377",6.26 cents
9,1870,38558371,"$3,421,198",8.87 cents


In [4]:
df2=htmltables[1]
df2

,0
0,"The U.S. population reached 300,000,000 on 17 ..."


The first table retrieved from the webpage contains the desired data. The data quality issues in the table include:
- The first row should be the column names
- The * character in the last row of the first column 
- The $ and , characters in the third column
- Values in the third column using words not numbers (ie. 4.5 billion instead of 450000000)
- Values in the forth column being described as both cents and dollars
- The $ character in the forth column
- The word 'cents' in the forth column 

In [5]:
# Replace the column names with the first row
colnames=df1.iloc[0]
df1=df1[1:]
df1.columns=colnames
df1.head()

,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents


## Create a function to cleanse the first column, test and apply

In [6]:
def cleanse_year(year):
    year=year.replace('*', '')
    return(year)

# Test funciton  
print(cleanse_year('1989*'))
print(cleanse_year('20*10'))

1989
2010


In [7]:
# Create new Dataframe for cleansed data
dfc=pd.DataFrame()
dfc["Census Year"] = df1["Census Year"].apply(cleanse_year)
dfc.head()

,Census Year
1,1790
2,1800
3,1810
4,1820
5,1830


Since the Total Population column requires no cleansing, it will be added to the cleansed dataframe as is

In [8]:
dfc['Total Population']=df1['Total Population']
dfc.head()

,Census Year,Total Population
1,1790,3929214
2,1800,5308483
3,1810,7239881
4,1820,9633822
5,1830,12866020


## Create a function to cleanse the third column, test, and apply

In [9]:
def cleanse_cost(cost):
    cost=cost.replace(',','')
    cost=cost.replace('$','')
    if 'Billion' in cost:
        cost=cost.replace(' Billion','')
        cost=str(int(float(cost)*1000000000))
    return(cost)

# Test the function
print(cleanse_cost('$1,000'))
print(cleanse_cost('$1.2 Billion'))

1000
1200000000


In [10]:
# Add the cleansed data to the new dataframe with .apply() 
dfc['Census Cost']=df1['Census Cost'].apply(cleanse_cost)
dfc.head()

,Census Year,Total Population,Census Cost
1,1790,3929214,44377
2,1800,5308483,66109
3,1810,7239881,178445
4,1820,9633822,208526
5,1830,12866020,378545


## Create a function to cleanse the forth column, test, and apply

In [11]:
def cleanse_avg(avg):
    if ' cents' in avg:
        avg=avg.replace(' cents','')
        avg=str(round(float(avg)/100,2))
    if '$' in avg:
        avg=avg.replace('$','')
    return(avg)

# Test the function
print(cleanse_avg('$4.76'))
print(cleanse_avg('17.07 cents'))

4.76
0.17


In [12]:
# Add the cleansed data to the new dataframe with .apply() 
dfc['Average Cost Per Person']=df1['Average Cost Per Person'].apply(cleanse_avg)
dfc.head()

,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,44377,0.01
2,1800,5308483,66109,0.01
3,1810,7239881,178445,0.02
4,1820,9633822,208526,0.02
5,1830,12866020,378545,0.03


## Save the cleansed dataframe as a csv

In [13]:
dfc.to_csv('census_cost_cleansed.csv',index=False)

## Read the csv and explore

In [14]:
cleansed_df=pd.read_csv('census_cost_cleansed.csv')
# .info() will display the type of data in each column
cleansed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Census Year              23 non-null     int64  
 1   Total Population         23 non-null     int64  
 2   Census Cost              23 non-null     int64  
 3   Average Cost Per Person  23 non-null     float64
dtypes: float64(1), int64(3)
memory usage: 864.0 bytes


In [15]:
# .describe() will display statistics on the dataframe 
cleansed_df.describe()

,Census Year,Total Population,Census Cost,Average Cost Per Person
count,23.0000,2.300000e+01,2.300000e+01,23.000000
mean,1900.0000,1.039211e+08,9.445587e+08,3.368261
std,67.8233,9.622629e+07,2.830037e+09,9.278399
min,1790.0000,3.929214e+06,4.437700e+04,0.010000
25%,1845.0000,2.013067e+07,1.128361e+06,0.055000
50%,1900.0000,7.630339e+07,1.185400e+07,0.170000
75%,1955.0000,1.653245e+08,1.096980e+08,0.655000
max,2010.0000,3.087455e+08,1.300000e+10,42.110000


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eb0a1b1e-7717-4570-b08b-c403cebb0472' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>